In [5]:
from sklearn.decomposition import PCA
from numpy import argpartition

import pandas as pd
import requests
import json
from collections import Counter
API_URL = 'http://data.uk.parlameter.si/v1'

In [10]:
session_id=2312
def getPercent(a, b, c, d=None):
    a = 0 if pd.isnull(a) else a
    b = 0 if pd.isnull(b) else b
    c = 0 if pd.isnull(c) else c
    if d:
        d = 0 if pd.isnull(d) else d
        devizer = float(sum([a, b, c, d]))
        if devizer:
            return max(a, b, c, d) / devizer * 100
        else:
            return 0
    else:
        devizer = float(sum([a, b, c]))
        if devizer:
            return max(a, b, c) / devizer * 100
        else:
            return 0


def getMPsList(row, proti):
    try:
        return json.dumps(list(proti[row.name].reset_index()['voter']))
    except:
        try:
            # fix if session has one vote
            return json.dumps(list(proti.values[0]))
        except:
            return json.dumps([])


def getPGsList(row, proti):
    try:
        pgs = [str(pg) for pg in list(proti[row.name].reset_index()['voterparty'])]
        return json.dumps(dict(Counter(pgs)))
    except:
        return json.dumps({})


def getPartyBallot(row):
    """
    using for set ballot of party:

    methodology: ignore not_present
    """
    stats = {'za': row['option_za'],
             'proti': row['option_proti'],
             'kvorum': row['option_kvorum'],
             'ni': row['option_ni']}
    if max(stats.values()) == 0:
        return '[]'
    max_ids = [key for key, val in stats.iteritems() if val == max(stats.values())]
    return json.dumps(max_ids)


def getIntraDisunion(row):
    maxOptionPercent = getPercent(row['option_za'],
                                  row['option_proti'],
                                  row['option_kvorum'])
    if maxOptionPercent == 0:
        return 0
    return 100 - maxOptionPercent


def getOptions(row, side):
    maxOptionPercent = getPercent(row['option_za'],
                                  row['option_proti'],
                                  row['option_kvorum'],
                                  row['option_ni'])
    stats = {'for': row['option_za'],
             'against': row['option_proti'],
             'abstain': row['option_kvorum'],
             'not_present': row['option_ni']}
    max_opt = max(stats, key=stats.get)
    max_ids = [key for key, val in stats.iteritems() if val == max(stats.values())]

    if len(max_ids) > 1:
        if 'not_present' in max_ids:
            max_ids.remove('not_present')
            if len(max_ids) > 1:
                max_vote = 'cant_compute'
            else:
                max_vote = max_ids[0]
        else:
            max_vote = 'cant_compute'
    else:
        max_vote = max_ids[0]

    outliers = []
    if side == 'oppo':
        # if side is oppozition don't show outliers
        pass
        #if max_vote != 'not_present':
        #    outliers = [opt for opt in ['for', 'against'] if stats[opt]]
    else:
        if max_vote != 'not_present':
            outliers = [opt for opt in ['abstain', 'for', 'against'] if stats[opt]]
    for opt in max_ids:
        if opt in outliers:
            outliers.remove(opt)

    return json.dumps({'votes': {
                                 'for': row['option_za'],
                                 'against': row['option_proti'],
                                 'abstain': row['option_kvorum'],
                                 'not_present': row['option_ni'],
                                 },
                       'max': {
                               'max_opt': max_vote,
                               'maxOptPerc': maxOptionPercent
                               },
                       'outliers': outliers})

In [16]:
url = API_URL + '/getVotesOfSessionTable/' + str(session_id) + '/'
print(url)
data = pd.read_json(url)
if data.empty:
    print "Empty"
coalition = requests.get(API_URL + '/getCoalitionPGs').json()['coalition']
#partys = Organization.objects.filter(classification='poslanska skupina')
partys = requests.get("http://data.uk.parlameter.si/v1/getAllPGsExt/").json()
paries_ids = partys.keys()
orgs = requests.get(API_URL + '/getAllPGsExt/')
data['option_ni'] = 0
data['option_za'] = 0
data['option_proti'] = 0
data['option_kvorum'] = 0
data.loc[data['option'] == 'ni', 'option_ni'] = 1
data.loc[data['option'] == 'aye', 'option_za'] = 1
data.loc[data['option'] == 'no', 'option_proti'] = 1
data.loc[data['option'] == 'tellno', 'option_kvorum'] = 1
data.loc[data['option'] == 'tellyes', 'option_kvorum'] = 1
data['voter_unit'] = 1
data['is_coalition'] = 0
data.loc[data['voterparty'].isin(coalition), 'is_coalition'] = 1

#za proti ni kvorum
all_votes = data.groupby('vote_id').sum()

all_votes['max_option_percent'] = all_votes.apply(lambda row: getPercent(row['option_za'], row['option_proti'], row['option_kvorum'], row['option_ni']), axis=1)

m_proti = data[data.option_proti == 1].groupby(['vote_id']).apply(lambda x: x["voter"])
m_za = data[data.option_za == 1].groupby(['vote_id']).apply(lambda x: x["voter"])
m_ni = data[data.option_ni == 1].groupby(['vote_id']).apply(lambda x: x["voter"])
m_kvorum = data[data.option_kvorum == 1].groupby(['vote_id']).apply(lambda x: x["voter"])

pg_proti = data[data.option_proti == 1].groupby(['vote_id']).apply(lambda x: x["voterparty"])
pg_za = data[data.option_za == 1].groupby(['vote_id']).apply(lambda x: x["voterparty"])
pg_ni = data[data.option_ni == 1].groupby(['vote_id']).apply(lambda x: x["voterparty"])
pg_kvorum = data[data.option_kvorum == 1].groupby(['vote_id']).apply(lambda x: x["voterparty"])

all_votes['m_proti'] = all_votes.apply(lambda row: getMPsList(row, m_proti), axis=1)
all_votes['m_za'] = all_votes.apply(lambda row: getMPsList(row, m_za), axis=1)
all_votes['m_ni'] = all_votes.apply(lambda row: getMPsList(row, m_ni), axis=1)
all_votes['m_kvorum'] = all_votes.apply(lambda row: getMPsList(row, m_kvorum), axis=1)

all_votes['pg_proti'] = all_votes.apply(lambda row: getPGsList(row, pg_proti), axis=1)
all_votes['pg_za'] = all_votes.apply(lambda row: getPGsList(row, pg_za), axis=1)
all_votes['pg_ni'] = all_votes.apply(lambda row: getPGsList(row, pg_ni), axis=1)
all_votes['pg_kvorum'] = all_votes.apply(lambda row: getPGsList(row, pg_kvorum), axis=1)

http://data.uk.parlameter.si/v1/getVotesOfSessionTable/2312/


In [17]:
all_votes['coal'] = data[data.is_coalition == 1].groupby(['vote_id']).sum().apply(lambda row: getOptions(row, 'coal'), axis=1)
all_votes['oppo'] = data[data.is_coalition == 0].groupby(['vote_id']).sum().apply(lambda row: getOptions(row, 'oppo'), axis=1)

In [18]:
data

,date,id,option,orgvoter,result,session_id,text,vote_id,voter,voterparty,option_ni,option_za,option_proti,option_kvorum,voter_unit,is_coalition
0,2018-07-19 14:28:00,1289915,aye,NaN,True,2312,Royal Assent — Independent Complaints and Grie...,1903,191,1,0,1,0,0,1,0
1,2018-07-19 14:28:00,1289916,aye,NaN,True,2312,Royal Assent — Independent Complaints and Grie...,1903,209,1,0,1,0,0,1,0
2,2018-07-19 14:28:00,1289917,tellaye,NaN,True,2312,Royal Assent — Independent Complaints and Grie...,1903,240,1,0,0,0,0,1,0
3,2018-07-19 14:28:00,1289918,aye,NaN,True,2312,Royal Assent — Independent Complaints and Grie...,1903,261,1,0,1,0,0,1,0
4,2018-07-19 14:28:00,1289919,aye,NaN,True,2312,Royal Assent — Independent Complaints and Grie...,1903,289,1,0,1,0,0,1,0
5,2018-07-19 14:28:00,1289920,aye,NaN,True,2312,Royal Assent — Independent Complaints and Grie...,1903,304,1,0,1,0,0,1,0
6,2018-07-19 14:28:00,1289921,aye,NaN,True,2312,Royal Assent — Independent Complaints and Grie...,1903,322,1,0,1,0,0,1,0
7,2018-07-19 14:28:00,1289922,aye,NaN,True,2312,Royal Assent — Independent Complaints and Grie...,1903,402,1,0,1,0,0,1,0
8,2018-07-19 14:28:00,1289923,aye,NaN,True,2312,Royal Assent — Independent Complaints and Grie...,1903,440,1,0,1,0,0,1,0
9,2018-07-19 14:28:00,1289924,aye,NaN,True,2312,Royal Assent — Independent Complaints and Grie...,1903,475,1,0,1,0,0,1,0
